# Generate 2D point cloud

Use this notebook to generate 2D point clouds from a PNG file

## Read and prepare the data

In [1]:
points_image_file = "images/dog_cat.png"

In [95]:
import cv2
import numpy as np
from PIL import Image

# Load the points image
points_image = cv2.imread(points_image_file, cv2.IMREAD_GRAYSCALE)

# Set important variables
height, width = points_image.shape
classes = np.unique(points_image)
classes_count = len(classes)-1

Extract all non 0 points from the image

In [55]:
points = []

for y in range(0, height):
    for x in range(0, width):
        pixel_color = points_image[y][x]
        if pixel_color > 0:
            points.append([x, height-y, pixel_color])

points = np.array(points)

## Generate random points in the point cloud

In [ ]:
# Random points parameters

points_to_add = 10000           # Number of random points to be added to the point cloud (if they meet the conditions)
min_distance_other_points = 5   # Minimum distance of a new point to an existing one
max_distance_center = width/2   # Maximum distance from the image cente

In [64]:
import random

# Compute the center of the image
center = np.array([height/2, width/2])

for i in range(0, points_to_add):
    # Create a random point
    p = np.random.randint(height, size=(1, 2))

    # Find the distance to the closest existing point
    dist = np.min(np.linalg.norm(points[:,0:2] - p, axis=1))

    # Check if the filter criteria are met and add the point if yes
    if dist >= min_distance_other_points and np.linalg.norm(center - p) < max_distance_center:
        points = np.append(points, [[p[0][0], p[0][1], 0]], axis=0)


## Display the point cloud

Common Plotly settings

In [61]:
import plotly.graph_objects as go

common_layout = go.Layout(
    autosize=False,
    width=width,
    height=height,
    margin=go.layout.Margin(l=10, r=10, b=10, t=10)
)

Display the point cloud using Plotly

In [62]:
data = go.Scatter(
    x=points[:,0], 
    y=points[:,1], 
    mode='markers',
)
go.Figure(data=data, layout=layout).show()

## Generate the features

Perform MSE minimization to find the best features that can be classified with randomly generated weights

Parameters for the feature optimization

In [227]:
features_count = 10
learning_rate = 0.001
iterations = 1000

Create random labels

In [228]:
W = 2*np.random.rand(features_count, classes_count) - 1

In [229]:
# Transform the weights to be mappable to an int between 0 and 9 (a signle digit)
W = (10 + 10*W)/2
W = (2*W.astype(np.int32) - 10) / 10.0

In [241]:
# Show the weights as digits
W_digits = (10+10*W)/2

print(np.array2string(W_digits.T.astype(np.int8),  separator=''))

[[0745096908]
 [0577334059]]


Prepare variables

In [230]:
points_count = points.shape[0]

# Initialize the features
X = 2*np.random.rand(points_count, features_count)-1

# One hot encoding of the labels
labels = np.zeros((points_count, classes_count))
for id, c in enumerate(classes[1:]):
    labels[:, id] = (points[:,2] == c)

Optimize the features

In [231]:
for i in range(0, iterations):
    predictions = X @ W

    diffs = labels - predictions

    loss = np.sum(diffs ** 2) / (classes_count * points_count)

    grad_X = np.zeros((points_count, features_count))
    for c in range(0, classes_count):
        grad_X -= 2 * np.expand_dims(diffs[:,c], 1) * np.expand_dims(W[:,c], 0)

    X -= learning_rate * grad_X

    print(f"Iteration {i}\tLoss = {loss}")

eration 539	Loss = 0.0006541892574732403
Iteration 540	Loss = 0.0006460126374619086
Iteration 541	Loss = 0.0006379391019078147
Iteration 542	Loss = 0.0006299673334374122
Iteration 543	Loss = 0.0006220960318657331
Iteration 544	Loss = 0.0006143239139651917
Iteration 545	Loss = 0.0006066497132376342
Iteration 546	Loss = 0.0005990721796895798
Iteration 547	Loss = 0.0005915900796106124
Iteration 548	Loss = 0.0005842021953548709
Iteration 549	Loss = 0.0005769073251255961
Iteration 550	Loss = 0.0005697042827626881
Iteration 551	Loss = 0.0005625918975332289
Iteration 552	Loss = 0.0005555690139249272
Iteration 553	Loss = 0.0005486344914424438
Iteration 554	Loss = 0.0005417872044065533
Iteration 555	Loss = 0.0005350260417561016
Iteration 556	Loss = 0.0005283499068527172
Iteration 557	Loss = 0.0005217577172882378
Iteration 558	Loss = 0.0005152484046948118
Iteration 559	Loss = 0.0005088209145576333
Iteration 560	Loss = 0.0005024742060302758
Iteration 561	Loss = 0.0004962072517525848
Iteration 562

## Display Results

In [242]:
class_to_show = 0

In [244]:
predicted = X @ W[:, class_to_show]
predicted_points = points[predicted > 0.5, :]

data = go.Scatter(
    x=predicted_points[:,0], 
    y=predicted_points[:,1], 
    mode='markers',
)

go.Figure(data=data, layout=common_layout).show()

## Export to CSV

In [11]:
data = np.concatenate((points[:, 0:2], X),1)

np.random.shuffle(data)

np.savetxt("html/points.csv", data, delimiter=",", fmt="%.3f", header="x,y,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9", comments='')